### Goal: Merge the data sets below into one clean data set that can be used to perform basic exploratory data analysis.

## Data Sets Used

**Fire Incidents (2010-2017)**
Catalogues all fire incidents to which the PFD was called. Why: Fire Incidents are what we will be trying to predict and therefore will be used to evaluate the accuracy of our model.
Peoria Fire Department

**Fire Inspections
(2010-2017)**
Catalogues inspection results for non-residential buildings Why: How long ago an inspection occured may impact the risk of a fire in the near future.
Peoria Fire Department

**Building Characteristics**
Catalogues several characteristics of all buildings, such as year built, square footage, etc. Why: Some building characteristics may impact the risk of a fire in the future. For example, older buildings may contain knob-and-tube electrical wiring, which is less safe than modern electrical wiring.
Peoria County GIS Department / City of Peoria Information Systems Department

_Optional_

**Code Violations**
(Dates?)
Catalogues ordinance violations found when inspecting buildings Why: Code violations may point to issues that could cause a fire, such as exposed electrical wiring.
City of Peoria Community Development Department

## My thought process

We have 5 data sets in 3 main categories that we'll be using for the exploratory data analysis (EDA) but they all have a variety of column names. In addition, our inspections data is split between 2010-2016 data and 2017 data, in spreadsheets that have different columns.

__Categories of datasets that we’re working with:__
1. Building Characteristics
2. Fire Inspections
3. Fire Incidents

__Names of the datasets that we're working with:__
1. commercial_building_list.csv - commercial buildings in the city with firezone id and firestation (Building Characteristics)
~~2. addresses_commercialbuildings_peoria.csv - characteristics of commercial buildings in Peoria  (Building Characteristics)~~
~~3. incidents_of_fire_2010_to_2017.csv - incidents of fire between the years of 2010-2017, pulled from ADSi tool (Fire Incidents)~~
4. inspection_data_2017.csv - fire inspections in 2017, pulled from current Peoria Fire Department tool (Fire Inspections)
~~5. inspections_data_before_2017.csv - fire inspections from 2010 to 2016 (Fire Inspections)~~


## Step 1

Merge the 3 data sets so that we have one final data set of all commercial buildings with inspection results and OVAP scores for 2017.


In [1]:
## 1. Import all 3 data sets to take a look at their columns

import pandas as pd
from pathlib import Path

## 1.a Set path for data sets

inspections_path = Path('.', 'data', 'inspection_data_2017.csv')
buildings_path = Path('.', 'data', 'commercial_building_list.csv')
OVAP_path = Path('.', 'data', 'commercial_with_OVAP_score.csv')

## 1.b Read in data to variables

inspections = pd.read_csv(inspections_path)
buildings = pd.read_csv(buildings_path)
OVAP = pd.read_csv(OVAP_path)

print(buildings.loc[:,'full_address'].head())
print(inspections.loc[:, 'Address1'].head())
print(OVAP.loc[:, 'Address'].head())

OVAP.head()


0        10405 N JULIET CT
1    10225 N KNOXVILLE AVE
2           1715 W ALTA RD
3           1601 W ALTA RD
4           1415 W ALTA RD
Name: full_address, dtype: object
0    4700 N University ST #60
1             100 State ST #A
2              1024 W MAIN ST
3        2607 N University ST
4           100 Alexander AVE
Name: Address1, dtype: object
0      8875 N Knoxville AVE
1          Peoria, IL 61615
2    2200 W War Memorial DR
3                    #DU19A
4          Peoria, IL 61613
Name: Address, dtype: object


,Occupancy Name,Address,NFF GPM,Flow Available,OVAP Score
0,Academy of Okinawan Karate,8875 N Knoxville AVE,0.0,Insufficient,0.00
1,NaN,"Peoria, IL 61615",NaN,NaN,NaN
2,the former Napoli's Pizza,2200 W War Memorial DR,196.0,Sufficient,29.46
3,NaN,#DU19A,NaN,NaN,NaN
4,NaN,"Peoria, IL 61613",NaN,NaN,NaN


In [2]:
## Remove unnecessary columns in buildings dataframe

buildings.drop(['street_number', 'street_directionality', 'street_name', 'street_type', 'owner_middleinitial'], axis=1, inplace=True)
buildings.columns


Index(['feature_ID', 'lower_PIN', 'PIN', 'owner_name', 'owner_address',
       'owner_address2', 'owner_city', 'owner_state', 'full_address', 'city',
       'state', 'zipcode', 'tax_code', 'first_name', 'last_name',
       'firestation', 'firezone'],
      dtype='object')

In [3]:
## Remove unnecessary columns in OVAP dataframe

OVAP.drop(['NFF GPM', 'Flow Available'], axis=1, inplace=True)
OVAP.columns

Index(['Occupancy Name', 'Address', 'OVAP Score'], dtype='object')

In [4]:
## Remove unnecessary columns in inspections dataframe - based on Roland's recommendations

inspections.drop(['Subcategory',
       'Email', 'Phone', 'FAX', 'OccupancyNotes', 'MapPage',
       'FireAlarmPanelLoc', 'MasterKeyLoc',
       'BasementPresent', 'Width', 'Length', 'GasLPGShutoffLoc',
       'ElectricalPanelLoc', 'FDConnections', 'SprinklerRoomLoc',
       'OtherLocInfo', 'WaterSupplyInfo', 'ExposureInfo', 'HazMat',
       'BuildingAccess', 'AccessProblems', 'VentilationProblems',
       'TimeNormallyOccupied', 'PrePlanNotes', 'UserOccupancyID',
       'LicenseNumber', 'NumberOfUnits', 'OccupancyLoad', 'UBCode', 'NFPACode',
       'IBCode', 'AssessorParcelNumber', 'NumberMilepost',
       'StreetPrefixDirection', 'StreetOrHighwayName', 'StreetType',
       'StreetSuffix', 'AptOrSuite', 'ISOID', 'Latitude',
       'BuildingNumber', 'Longitude',
       'AssessedValue', 'NationalGrid',
       'UtilityBillingNumber', 'RoofTypeId', 'RoofMaterialId',
       'RoofConstructionId', 'StationID', 'DispatchID',
       'Population DensityID'], axis=1, inplace=True)
inspections.columns

Index(['Name', 'Address1', 'Address2', 'City', 'State', 'Zip', 'OccupancyType',
       'BuildingHeight', 'NumberOfFloors', 'SquareFeet',
       'CriticalInfrastructure', 'YearBuilt', 'NFIRSPropertyUse',
       'ConstructionTypeId'],
      dtype='object')

In [5]:
## 2. OVAP Score data set contains extra rows with City, State, ZIP and Unit information. We need to remove City, State,
## and ZIP. Then add a column to the row with the address that will denote the unit.

## 2.a Add a "Unit" column to the OVAP data set.

OVAP.insert(2, 'Unit', 'NaN')
OVAP.head()

,Occupancy Name,Address,Unit,OVAP Score
0,Academy of Okinawan Karate,8875 N Knoxville AVE,NaN,0.00
1,NaN,"Peoria, IL 61615",NaN,NaN
2,the former Napoli's Pizza,2200 W War Memorial DR,NaN,29.46
3,NaN,#DU19A,NaN,NaN
4,NaN,"Peoria, IL 61613",NaN,NaN


In [6]:
## 2.b Write a for loop to move unit information from 'Address' column to previous row's 'Unit' column.

## Find number of rows in OVAP
OVAP.shape

## Create dummy copy of OVAP to play with
OVAP_test = OVAP

## Go through all rows in OVAP
for row in range(0, 2422):
    
    ## If row's Occupancy Name is a str, skip this row
    if type(OVAP_test.loc[row, 'Occupancy Name']) == str:
        pass
    
    ## Else, check to see if '#' is in the Address column of this row
    else:
        if '#' in OVAP_test.loc[row, 'Address']:
            
            ## If so, this is a Unit number and is moved to the previous row's Unit column.
            ## Then, drop this column.
            OVAP_test.loc[row-1, 'Unit'] = OVAP_test.loc[row, 'Address']
            OVAP_test.drop(row, inplace=True)
            
        ## Else, this is City, State, and ZIP and is not needed, so it is dropped.    
        else:
            OVAP_test.drop(row, inplace=True)
            
OVAP.head(20)
        

,Occupancy Name,Address,Unit,OVAP Score
0,Academy of Okinawan Karate,8875 N Knoxville AVE,NaN,0.00
2,the former Napoli's Pizza,2200 W War Memorial DR,#DU19A,29.46
5,(Office w/ Unkown name),701 Main ST,#7,29.46
8,(vacant mercantile),2200 W War Memorial DR,#DU08,21.02
11,1505 on the Avenue Apartments,1505 N Peoria AVE,NaN,39.11
13,311 SW Water,311 SW Water ST,NaN,33.00
15,401 Management Office,401 SW Water ST,#201,31.53
18,401 Water - CAFE 401,401 SW Water ST,#102,30.31
22,401 Water - Hodges Loizzi Eisenhammer,401 SW Water ST,#106,30.55
25,401 Water - VOLT,401 SW Water ST,#101,27.13


In [7]:
## Capitalize all addresses in inspections and OVAP_test dataframe to prepare for joining with buildings

## OVAP_test capitalization
OVAP_test['Address'] = OVAP_test['Address'].str.upper()
print(OVAP_test.head())

## inspections capitalization
inspections['Address1'] = inspections['Address1'].str.upper()
print(inspections.head())

                   Occupancy Name                 Address    Unit  OVAP Score
0      Academy of Okinawan Karate    8875 N KNOXVILLE AVE     NaN        0.00
2       the former Napoli's Pizza  2200 W WAR MEMORIAL DR  #DU19A       29.46
5         (Office w/ Unkown name)             701 MAIN ST      #7       29.46
8             (vacant mercantile)  2200 W WAR MEMORIAL DR   #DU08       21.02
11  1505 on the Avenue Apartments       1505 N PEORIA AVE     NaN       39.11
                          Name                  Address1  Address2    City  \
0                  50's Dinner  4700 N UNIVERSITY ST #60       NaN  Peoria   
1               8 Bit Beercade           100 STATE ST #A       NaN  Peoria   
2             A WORLD OF WINGS            1024 W MAIN ST       NaN  PEORIA   
3                   Agatucci's      2607 N UNIVERSITY ST       NaN  Peoria   
4  Alexander Street Steakhouse         100 ALEXANDER AVE       NaN  Peoria   

  State    Zip         OccupancyType  BuildingHeight  NumberOfF

In [8]:
## 3. Join OVAP_test and buildings dataframes by the Address column

## 3.1 Rename buildings and inspections address columns to 'Address'

buildings.rename(columns={'full_address':'Address'}, inplace=True)
inspections.rename(columns={'Address1':'Address'}, inplace=True)


## 3.2 Join buildings and OVAP_test dataframes on shared 'Address' column

buildings_with_OVAP = pd.merge(buildings, OVAP_test)
buildings_with_OVAP.head()

## 3.3 Join buildings_with_OVAP and inspections on shared 'Address' column
joined_data = pd.merge(buildings_with_OVAP, inspections)

## Resulting buildings_with_OVAP need to be cleaned
print(joined_data.shape)
joined_data

(3735, 33)


,feature_ID,lower_PIN,PIN,owner_name,owner_address,owner_address2,owner_city,owner_state,Address,city,...,State,Zip,OccupancyType,BuildingHeight,NumberOfFloors,SquareFeet,CriticalInfrastructure,YearBuilt,NFIRSPropertyUse,ConstructionTypeId
0,4,929378003,929378003,MARK BAYLEY FAMILY LLC,C/O M & B,P O BOX 385,CARMI,IL,1415 W ALTA RD,PEORIA,...,IL,61615,B- Business,30.0,1.0,4500.0,NaN,NaN,571,Ordinary (Joisted Masonry) Class III
1,5,929378003,929378003,MARK BAYLEY FAMILY LLC,C/O M & B,P O BOX 385,CARMI,IL,1415 W ALTA RD,PEORIA,...,IL,61615,B- Business,30.0,1.0,4500.0,NaN,NaN,571,Ordinary (Joisted Masonry) Class III
2,11,929104018,929104018,MAC'S CONVENIENCE STORES LLC,4080 JONATHON MOORE PIKE,,COLUMBUS,IN,1514 W HICKORY GROVE RD,DUNLAP,...,IL,61615,M-Motor Vehicle Service Stations,25.0,1.0,15000.0,NaN,NaN,571,Ordinary (Joisted Masonry) Class III
3,15,1420476024,1420476024,KROGER LIMITED PARTNERSHIP I,PROPERTY TAX 7TH FLOOR,1014 VINE ST,CINCINNATI,OH,801 W LAKE AVE,PEORIA,...,IL,61614,B- Business,NaN,1.0,0.0,NaN,NaN,500,Ordinary (Joisted Masonry) Class III
4,15,1420476024,1420476024,KROGER LIMITED PARTNERSHIP I,PROPERTY TAX 7TH FLOOR,1014 VINE ST,CINCINNATI,OH,801 W LAKE AVE,PEORIA,...,IL,61614,B- Business,NaN,NaN,NaN,NaN,NaN,NaN,Ordinary (Joisted Masonry) Class III
5,15,1420476024,1420476024,KROGER LIMITED PARTNERSHIP I,PROPERTY TAX 7TH FLOOR,1014 VINE ST,CINCINNATI,OH,801 W LAKE AVE,PEORIA,...,IL,61614,B- Business,NaN,NaN,NaN,NaN,NaN,581,Ordinary (Joisted Masonry) Class III
6,15,1420476024,1420476024,KROGER LIMITED PARTNERSHIP I,PROPERTY TAX 7TH FLOOR,1014 VINE ST,CINCINNATI,OH,801 W LAKE AVE,PEORIA,...,IL,61614,B- Business,NaN,2.0,48000.0,NaN,NaN,519,Ordinary (Joisted Masonry) Class III
7,15,1420476024,1420476024,KROGER LIMITED PARTNERSHIP I,PROPERTY TAX 7TH FLOOR,1014 VINE ST,CINCINNATI,OH,801 W LAKE AVE,PEORIA,...,IL,61614,B- Business,NaN,NaN,NaN,NaN,NaN,NaN,Ordinary (Joisted Masonry) Class III
8,15,1420476024,1420476024,KROGER LIMITED PARTNERSHIP I,PROPERTY TAX 7TH FLOOR,1014 VINE ST,CINCINNATI,OH,801 W LAKE AVE,PEORIA,...,IL,61614,B- Business,NaN,1.0,0.0,NaN,NaN,500,Ordinary (Joisted Masonry) Class III
9,15,1420476024,1420476024,KROGER LIMITED PARTNERSHIP I,PROPERTY TAX 7TH FLOOR,1014 VINE ST,CINCINNATI,OH,801 W LAKE AVE,PEORIA,...,IL,61614,B- Business,NaN,NaN,NaN,NaN,NaN,NaN,Ordinary (Joisted Masonry) Class III


In [9]:
joined_data.columns

Index(['feature_ID', 'lower_PIN', 'PIN', 'owner_name', 'owner_address',
       'owner_address2', 'owner_city', 'owner_state', 'Address', 'city',
       'state', 'zipcode', 'tax_code', 'first_name', 'last_name',
       'firestation', 'firezone', 'Occupancy Name', 'Unit', 'OVAP Score',
       'Name', 'Address2', 'City', 'State', 'Zip', 'OccupancyType',
       'BuildingHeight', 'NumberOfFloors', 'SquareFeet',
       'CriticalInfrastructure', 'YearBuilt', 'NFIRSPropertyUse',
       'ConstructionTypeId'],
      dtype='object')

In [11]:
## Just for fun - look at all entries in firezone 1112
joined_data.loc[joined_data.firezone.isin(['1112']), :].shape

(209, 33)